<a href="https://colab.research.google.com/github/hotbread213/createClass/blob/master/3_High_frequency_volatility_prediction_using_RNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IVADO Workshop Day 3 - Deep Learning

In this notebook, we will try to predict future price volatility of cryptocurrencies using a simple LSTM network using Keras. At the end, we will try to reproduce what we have done, but with the PyTorch library.

In [0]:
import time
import pandas as pd
import numpy as np
import requests
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

We will start by fetching some 1 min. price and volume data from a cryptocurrency exchange

In [0]:
import requests

market = 'BTCUSDT'
tick_interval = '1m'

n = 20

endTime = int(time.time()*1000)
startTime = endTime - n*60000*1000

ohlc = []
for i in range(200):
    url = 'https://api.binance.com/api/v1/klines?limit=1000&symbol='+market+\
    '&interval='+tick_interval+\
    '&endTime='+str(startTime+(i+1)*60000*1000)+\
    '&startTime='+str(startTime+i*60000*1000)
    
    data = requests.get(url).json()
    ohlc = ohlc + data


We have a dataset composed of approximately 20000 prices and volumes

In [0]:
len(ohlc)

In [0]:
df = pd.DataFrame(ohlc)
df = df.iloc[:,:6].astype(np.float32)
df.columns = ['time','open','high','low','close','volume']
df.head()

Let's create some features and a target

In [0]:
df['mean_price'] = df[['open','high','low','close']].mean(axis=1)

In [0]:
df['last_60_vol'] = df.mean_price.pct_change().rolling(60).std()*100

In [0]:
df['next_60_vol'] = df.mean_price.pct_change().iloc[::-1].rolling(60).std().iloc[::-1]*100

In [0]:
df['norm_volume']=(df.volume-df.volume.rolling(60).mean())/df.volume.rolling(60).std()

In [0]:
df.dropna(inplace=True)

We shape some examples that we are going to feed into an LSTM network

In [0]:
def make_examples(X, y):
    n = len(X)
    
    examples=[]
    targets=[]
    
    for i in range(n//5):
        examples.append(X[i*5:(i+1)*5])
        targets.append(y[(i+1)*5])
        
    return examples, targets

In [0]:
X = df[['last_60_vol','norm_volume']].values
y = df.next_60_vol.values

In [0]:
X, y = make_examples(X, y)

In [0]:
X = np.stack(X)
y = np.stack(y)

In [0]:
split = int(0.7*len(X))

In [0]:
X_train, y_train = X[:split], y[:split]
X_valid, y_valid = X[split:], y[split:]

Finally, let's fit a simple Keras neural network on our data

In [0]:
model = Sequential()

model.add(LSTM(16,input_shape=(5,2,)))
model.add(Dense(16,activation='relu'))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100)

In [0]:
plt.figure(figsize=(12,8))
plt.plot(y_valid[:100], label='Real')
plt.plot(model.predict(X_valid)[:100], label='Predicted')
plt.legend()
plt.title('Real vs. Predicted volatility on the validation set')

# Reproduce in PyTorch